In [25]:
import numpy as np
np.random.seed(1)

from keras.datasets import mnist
(x_train,y_train),(x_test,y_test) = mnist.load_data()

In [26]:
images , labels = (
    x_train[:1000].reshape(1000,28*28)/255,
    y_train[:1000]
)

one_hot_label = np.zeros((len(labels),10))

for i,l in enumerate(labels):
    one_hot_label[i][l] = l
labels = one_hot_label

In [27]:
test_images = x_test.reshape(len(x_test),28*28)/255
test_labels = np.zeros((len(y_test),10))

for i,l in enumerate(y_test):
    test_labels[i][l] = l

In [28]:
def tanh(x):
    return np.tanh(x)

def tanh2deriv(x):
    return 1- x*2

def softmax(x):
    temp = np.exp(x)
    return temp / np.sum(temp, axis=1, keepdims= True)

In [29]:
alpha, iterations = (2,1)
pixels_per_image, num_labels = (784,10)
batch_size = 128

input_rows, input_cols = (28,28)
kernel_rows, kernel_cols = (3,3)
num_kernels = 16

In [30]:
hidden_size = ((input_rows-kernel_rows)*(input_cols-kernel_cols))*num_kernels
kernels = 0.02*np.random.random((kernel_cols*kernel_rows,num_kernels)) - 0.1
weights_1_2 = 0.02*np.random.random((hidden_size,num_labels))-0.1

In [31]:
def get_image_section(layer, row_from, row_to, col_from, col_to):
    section = layer[:,row_from:row_to,col_from:col_to]
    return section.reshape(-1,1,row_to-row_from,col_to-col_from)

In [33]:
for j in range(iterations):
    correct_cnt = 0
    for i in range(1):#int(len(images)/batch_size)):
        batch_start, batch_end = ((i*batch_size),((i+1)*batch_size))
        layer_0 = images[batch_start:batch_end]
        layer_0 = layer_0.reshape(layer_0.shape[0],28,28)
    
        sects = list()
        for row_start in range(layer_0.shape[1]-kernel_rows):
            for col_start in range(layer_0.shape[2]-kernel_cols):
                sect = get_image_section(
                    layer_0,
                    row_start,
                    row_start+kernel_rows,
                    col_start,
                    col_start+kernel_cols
                )
                sects.append(sect)
        expanded_input = np.concatenate(sects,axis=1)  
        es = expanded_input.shape
        print(es,np.array(sects).shape)
        flattened_input = expanded_input.reshape(es[0]*es[1],-1)
        
        kernel_output = flattened_input.dot(kernels)
        layer_1 = tanh(kernel_output.reshape(es[0],-1))
        dropout_mask = np.random.randint(2,size=layer_1.shape)
        layer_1*=dropout_mask
        layer_2 = softmax(np.dot(layer_1,weights_1_2))


(128, 625, 3, 3) (625, 128, 1, 3, 3)
